
# 🌼 RAG기법의 이해와 적용 - 3차시(24.12.02)

---

In [97]:
from langchain.memory import ConversationBufferMemory
# 랭체인의 메모리 클래스 BufferMemory
# input 질문자의 응답
# output 챗봇의 응답
# 선입 선출 

In [ ]:
def coversation_chain(memory, query):
    # 대화 기록 출력
    chat_history_data = memory.load_memory_variables()
    print("Chat history data:", chat_history_data)
    
    chat_history = RunnablePassthrough.assign(
        chat_history=RunnableLambda(chat_history_data) | itemgetter(memory.memory_key)
    )
    
    llm = ChatOpenAI(model='gpt-4o', temperature=0)
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessage(content='너는 진철한 AI 비서야'),
            MessagesPlaceholder(variable_name='chat_history'),
            HumanMessage(content='{input}')
        ]
    )
    
    chain = chat_history | prompt | llm | StrOutputParser()

    # 체인 구조 출력
    print("Chain structure:", chain)

    try:
        answer = chain.invoke({'input': query})
        print("AI Response:", answer)
    except TypeError as e:
        print("Error details:", e)
        raise  # 오류를 다시 발생시켜서 확인
    
    memory.save_context(inputs={'human': query}, outputs={'ai': answer})
    return answer


In [98]:
memory = ConversationBufferMemory()

In [99]:
memory.save_context(
    #명시적으로 inpeu output 넣어줄 수 있음
    inputs = {
        'human':'안녕하세요. 휴대폰을 구매하러 왔어요'
    },
    outputs={
        'ai':'안녕하세요. 무엇을 찾으세요?'
    }
)

In [100]:
memory.load_memory_variables({})

{'history': 'Human: 안녕하세요. 휴대폰을 구매하러 왔어요\nAI: 안녕하세요. 무엇을 찾으세요?'}

In [101]:
from dotenv import load_dotenv

load_dotenv()

True

In [102]:
from langchain_teddynote import logging
logging.langsmith("LangChainProject")

LangSmith 추적을 시작합니다.
[프로젝트명]
LangChainProject


In [103]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

In [104]:
model = ChatOpenAI(model = 'gpt-4o')

In [105]:
prompt = ChatPromptTemplate.from_messages(
    # 대화형 프롬프트 생성
    [
        ('system','너는 친절한 챗봇이야'),
        MessagesPlaceholder(variable_name='chat_history'),
        # 이전 대화를 내용을 담을 자리를 만들어 줌
        ('human', '{input}')
        # 동적으로 담을 수 있도록 input으로
    ]
)

In [106]:
# 대화 메모리 생성
memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')
# 메세지 형태로 반환하겠다

In [107]:
memory.load_memory_variables({})

{'chat_history': []}

In [108]:
runnable = RunnablePassthrough.assign(
    chat_history = RunnableLambda(memory.load_memory_variables) | itemgetter('chat_history')
    # 메모리에서 데이터를 로드하고 chat_history 만 추출하겠다
    # input 내용 중에서도 chat_history값만 가져오기 => 대화이력 (깜끔한 대화 이력만 가져오므로 전처리 과정라 할 수 있음)
    # runnable 입력 데이터를 처리한다
)

In [109]:
chain = runnable | prompt | model

In [110]:
respone = chain.invoke({'input' :'안녕하세요. 제 이름은 아무개 입니다.'})
print(respone.content)

안녕하세요, 아무개님! 만나서 반갑습니다. 오늘 어떻게 도와드릴까요?


In [111]:
memory.save_context({'human' : '안녕하세요. 제 이름은 아무개 입니다.'}, {'ai' : respone.content})

In [112]:
respone = chain.invoke({'input' :'안녕하세요. 제 이름이 뭐라고 했죠?'})
print(respone.content)

당신의 이름은 "아무개"라고 하셨습니다. 맞나요?


In [113]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferWindowMemory

In [114]:
memory = ConversationBufferWindowMemory(k=1, return_messages=True, memory_key='chat_history')
# 메모리 초기화
# WindowMemory : 최근 대화만 저장
# BufferMemory : 순차적으로 모두 저장

In [115]:
def conversation_chain(memory, query):
    chat_history = RunnablePassthrough.assign(
        chat_history = RunnableLambda(memory.load_memory_variables) | itemgetter(memory.memory_key)
    )
    llm = ChatOpenAI(model = 'gpt-4o', temperature = 0)
    prompt = ChatPromptTemplate.from_messages(
        [
            ('system', '너는 친절한 AI 비서야'),
            MessagesPlaceholder(variable_name='chat_history'),
            ('human' , "{input}")
        ]
    )
    chain = chat_history | prompt | llm | StrOutputParser()
    # StrOutputParser() : 출력되는 값을 자동으로 파싱(불필요한 개행문자, 특수기호 등을 자동으로 처리)
    answer = chain.invoke({'input' : query})
    memory.save_context(inputs = {'human' : query}, outputs={'ai' : answer})
    return answer

In [116]:
response = conversation_chain(
    memory = memory,
    query = '갑자기 날씨가 너무 추워졌어. 이 때 먹을만한 음식을 추천해줘'
)
print('al : ', response)

al :  추운 날씨에는 따뜻하고 포근한 음식을 먹으면 몸도 마음도 따뜻해지죠. 몇 가지 추천해드릴게요:

1. **김치찌개**: 매콤하고 뜨거운 국물이 몸을 따뜻하게 해줍니다. 돼지고기나 참치 등을 넣어 다양하게 즐길 수 있어요.

2. **된장찌개**: 구수한 맛이 일품인 된장찌개는 두부, 감자, 호박 등을 넣어 푸짐하게 끓이면 좋습니다.

3. **삼계탕**: 영양가가 풍부한 삼계탕은 몸을 따뜻하게 해주고 기운을 북돋아 줍니다.

4. **칼국수**: 뜨거운 국물에 쫄깃한 면발이 어우러져 추운 날씨에 제격입니다.

5. **호떡**: 달콤한 시럽이 들어간 호떡은 간식으로 먹기 좋고, 손을 따뜻하게 해줍니다.

6. **붕어빵**: 길거리에서 쉽게 찾을 수 있는 붕어빵은 추운 날씨에 따뜻한 간식으로 제격입니다.

이 외에도 따뜻한 차나 커피와 함께 즐기면 더욱 좋겠죠. 따뜻하게 입고 건강 조심하세요!


In [117]:
response = conversation_chain(
    memory = memory,
    query = '그럼이럴 때 들을만한 노래를 추천해줘'
)
print('al : ', response)

al :  추운 날씨에 듣기 좋은 노래들은 따뜻한 분위기를 더해주거나 감성을 자극하는 곡들이 많습니다. 몇 가지 추천해드릴게요:

1. **아이유 - "밤편지"**: 잔잔하고 따뜻한 멜로디가 마음을 편안하게 해줍니다.

2. **폴킴 - "모든 날, 모든 순간"**: 감미로운 목소리와 멜로디가 추운 날씨에 잘 어울립니다.

3. **김광석 - "어느 60대 노부부 이야기"**: 감성적인 가사와 멜로디가 마음을 따뜻하게 해줍니다.

4. **에픽하이 - "춥다 (feat. 이하이)"**: 제목처럼 추운 날씨에 듣기 좋은 곡입니다.

5. **박효신 - "눈의 꽃"**: 겨울과 잘 어울리는 발라드로, 감성을 자극합니다.

6. **Carpenters - "Top of the World"**: 밝고 경쾌한 멜로디가 기분을 좋게 만들어 줍니다.

7. **Norah Jones - "Come Away With Me"**: 부드러운 재즈풍의 곡으로, 따뜻한 커피 한 잔과 함께 듣기 좋습니다.

이 노래들이 추운 날씨에 조금이나마 따뜻함을 더해주길 바랍니다. 즐겁게 감상하세요!


In [118]:
response = conversation_chain(
    memory = memory,
    query = '아까 추천해준 음식중에 하나만 선택해줘'
)
print('al : ', response)

al :  아직 음식에 대한 추천을 드린 적이 없는데요. 혹시 특정한 종류의 음식을 원하신다면 말씀해 주시면 추천해드리겠습니다! 예를 들어, 한식, 양식, 디저트 등 어떤 종류의 음식을 찾고 계신가요?


In [119]:
memory.load_memory_variables({})['chat_history']

[HumanMessage(content='아까 추천해준 음식중에 하나만 선택해줘', additional_kwargs={}, response_metadata={}),
 AIMessage(content='아직 음식에 대한 추천을 드린 적이 없는데요. 혹시 특정한 종류의 음식을 원하신다면 말씀해 주시면 추천해드리겠습니다! 예를 들어, 한식, 양식, 디저트 등 어떤 종류의 음식을 찾고 계신가요?', additional_kwargs={}, response_metadata={})]

In [120]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory

In [121]:
llm = ChatOpenAI(model = 'gpt-4o-mini')

In [122]:
memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit = 200,
    return_messages =True
)

In [123]:
memory.save_context(
    inputs={"human": "이 식당에서 가장 인기 있는 메뉴는 무엇인가요?"},
    outputs={
        "ai": "이 식당에서 가장 인기 있는 메뉴는 불고기 정식, 해물파전, 비빔밥, 그리고 감자탕입니다. 특히 불고기는 달콤하고 짭짤한 맛으로 외국인 손님들에게도 큰 인기를 끌고 있습니다."
    },
)
memory.save_context(
    inputs={"human": "채식주의자를 위한 메뉴가 제공되나요?"},
    outputs={
        "ai": "네, 채식주의자를 위한 메뉴로 채소 비빔밥, 두부구이, 야채전, 그리고 나물 반찬 세트를 제공합니다. 신선한 제철 채소로 만들어져 건강하고 맛있는 식사를 즐기실 수 있습니다."
    },
)
memory.save_context(
    inputs={"human": "어린이를 위한 메뉴도 있나요?"},
    outputs={
        "ai": "네, 어린이를 위한 메뉴로 미니 김밥, 떡볶이, 그리고 달콤한 간장 치킨을 준비하고 있습니다. 아이들이 좋아할 만한 맛과 건강을 고려한 요리들입니다."
    },
)
memory.save_context(
    inputs={"human": "이 식당은 어떤 분위기를 가지고 있나요?"},
    outputs={
        "ai": "이 식당은 한옥 스타일의 인테리어로 전통적인 한국의 분위기를 느낄 수 있습니다. 따뜻하고 편안한 조명과 현대적인 디자인이 조화를 이루어 가족 단위 손님뿐만 아니라 연인들의 데이트 장소로도 인기가 많습니다."
    },
)

In [124]:
memory.load_memory_variables({})['history']

[SystemMessage(content='The human asks about the most popular menu items at the restaurant. The AI responds that the most popular items are bulgogi, seafood pancake, bibimbap, and potato soup, noting that bulgogi is particularly popular among foreign guests due to its sweet and savory flavor. The human then inquires if there are menu options for vegetarians, and the AI confirms that there are vegetarian options available, including vegetable bibimbap, grilled tofu, vegetable pancakes, and a set of seasonal vegetable side dishes, all made with fresh ingredients for a healthy and delicious meal.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='어린이를 위한 메뉴도 있나요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='네, 어린이를 위한 메뉴로 미니 김밥, 떡볶이, 그리고 달콤한 간장 치킨을 준비하고 있습니다. 아이들이 좋아할 만한 맛과 건강을 고려한 요리들입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='이 식당은 어떤 분위기를 가지고 있나요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='이

In [125]:
memory.load_memory_variables({})['history'][0].content

'The human asks about the most popular menu items at the restaurant. The AI responds that the most popular items are bulgogi, seafood pancake, bibimbap, and potato soup, noting that bulgogi is particularly popular among foreign guests due to its sweet and savory flavor. The human then inquires if there are menu options for vegetarians, and the AI confirms that there are vegetarian options available, including vegetable bibimbap, grilled tofu, vegetable pancakes, and a set of seasonal vegetable side dishes, all made with fresh ingredients for a healthy and delicious meal.'